In [1]:
from geneticalgorithm import geneticalgorithm as ga
from ipynb.fs.full.fitness import *
from ipynb.fs.full.CSP_filter_data import *
import numpy as np
from sklearn.decomposition import PCA
from ipynb.fs.full.poincareSection import *
import pandas as pd
import scipy
from scipy.stats import variation 
import warnings
from sklearn.model_selection import KFold
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.svm import SVC
from ipynb.fs.full.Evaluated import *

fvalbest = 1
FES = 0
chvalbest = []
ACCVAL = []
ACCTRAIN = []
ACCTEST = []

def fitness(particle):
    
    np.seterr(divide='ignore', invalid='ignore')
    warnings.simplefilter("ignore", np.ComplexWarning)
    
    particle = [int(x) for x in particle]
    particle[0] = 5
    
    global FES 
    print(f"FES  = {FES}")

    # Calculate the total weight of the dimensions
    sumw = 0
    weight_start_index = 2
    for dim in range(int(particle[0])-1):
        sumw = sumw + particle[weight_start_index]
        weight_start_index = weight_start_index + 1

    f_number = 0
    p = 0

    #filter data using csp function
    fdata, label = CSP_filter_data(xdata,label1)

    total_p = 0
    for data in range(0, fdata.shape[1] - sumw, particle[1]):
        total_p += 1
    
    number_of_line = 4
    number_of_statistical_feature = 17
    feature = np.empty((fdata.shape[2], fdata.shape[0] * (2 * number_of_line) * number_of_statistical_feature))
    
    for segment in range(fdata.shape[2]):  #segment 
        for channel in range(fdata.shape[0]):  #channel
            phase_space = np.empty((total_p, particle[0]))
            for data in range(0, fdata.shape[1] - sumw, particle[1]):
                phase_space[p, 0] = fdata [channel, data, segment]
                next_data = 0
                weight_start_index = 2
                
                for dim in range(1,particle[0]):
                    next_data = next_data + particle[weight_start_index]
                    phase_space[p,dim] = fdata[channel, data + next_data, segment] 
                    weight_start_index = weight_start_index + 1 
                    
                p = p + 1
                
            n_components=2
            pca = PCA(n_components)
            phase_space_pca = pca.fit_transform(phase_space)
            
            minps = np.ones(len(phase_space_pca[0]))
            mxps = np.ones(len(phase_space_pca[0]))
            for norm in range(len(phase_space_pca[0])):
                minps[norm] = min(phase_space_pca[:, norm])
                mxps[norm] = max(phase_space_pca[:, norm])

            phase_space_pca_norm = np.empty((total_p, len(phase_space_pca[0])))
            for norm in range(len(phase_space_pca[0])):
                if minps[norm] == mxps[norm]:
                    phase_space_pca_norm[:, norm] = phase_space_pca[:, norm] - phase_space_pca[:, norm]
                else:
                    phase_space_pca_norm[:, norm] = (((phase_space_pca[:, norm] - minps[norm]) / (mxps[norm] - minps[norm])) -0.5 ) * 2 
            
            a1 = np.ones(number_of_line)
            b1 = np.ones(number_of_line)
            c1 = np.ones(number_of_line)
            
            # line y = 0
            a1[0] = 0 
            b1[0] = 1
            c1[0] = 0

            # line x = c
            a1[1] = 1 
            b1[1] = 0
            c1[1] = (particle[8] - 1) * (2) / (15) + (-1) # normalize between [-1,1]
       
            # line y = (a/b) * x + c
            a1[2] = (particle[6] - 1) * (2) / (15) + (-1) # normalize between [-1,1]
            b1[2] = (particle[7] - 1) * (2) / (15) + (-1) # normalize between [-1,1]
            c1[2] = 0
        
            # line y = -(a/b) * x + c
            a1[3] = -(particle[6] - 1) * (2) / (15) + (-1) # normalize between [-1,1]
            b1[3] = (particle[7] - 1) * (2) / (15) + (-1) # normalize between [-1,1]
            c1[3] = 0

            IntersectionPoint = 0 
            for line_number in range(len(a1)):
                px = np.empty(0)
                py = np.empty(0)
                for data_number in range(len(phase_space_pca_norm)-1):
                    x_axis_value, y_axis_value, pbool = poincareSection(phase_space_pca_norm[data_number, 0], phase_space_pca_norm[data_number, 1], phase_space_pca_norm[data_number + 1, 0], phase_space_pca_norm[data_number + 1, 1], a1[line_number], b1[line_number], c1[line_number])
                    
                    if pbool == 1:
                        #IntersectionPoint = IntersectionPoint + 1
                        px = np.append(px, x_axis_value)
                        py = np.append(py, y_axis_value)  

                if len(px) == 0 :
                    px = np.array([0, 0])
                    py = np.array([0, 0.1])
                
                px = [float(x) for x in px]
                feature[segment, f_number] = max(px)- min(px) #range
                feature[segment, f_number + 1] = np.quantile(px, 0.13) #quantile
                feature[segment, f_number + 2] = np.quantile(px, 0.25) - np.quantile(px, 0.75) #interquartile
                px_series = pd.Series(px)
                counts = px_series.value_counts()
                feature[segment, f_number + 3] = scipy.stats.entropy(counts)  #shannon entropy
                feature[segment, f_number + 4] = np.sqrt(np.mean(np.power(px, 2))) #root mean squared
                feature[segment, f_number + 5] = variation(px) #Coefficient of Variation
                feature[segment, f_number + 6] = pow(sum(px),2) #energy
                feature[segment, f_number + 7] = np.var(px) #variance
            
                feature[segment, f_number + 8] = max(py)- min(py) #range
                feature[segment, f_number + 9] = np.quantile(py, 0.13) #quantile
                feature[segment, f_number + 10] = np.quantile(py, 0.25) - np.quantile(py, 0.75) #interquartile
                py_series = pd.Series(py)
                counts = py_series.value_counts()
                feature[segment, f_number + 11] = scipy.stats.entropy(counts)  #shannon entropy
                feature[segment, f_number + 12] = np.sqrt(np.mean(np.power(py, 2))) #root mean squared
                feature[segment, f_number + 13] = variation(py) #Coefficient of Variation
                feature[segment, f_number + 14] = pow(sum(py),2) #energy
                feature[segment, f_number + 15] = np.var(py) #variance
                feature[segment, f_number + 16] = len(px)

                f_number = f_number + 17
                del px, py
                IntersectionPoint = 1

            del phase_space, phase_space_pca, pca, phase_space_pca_norm
            p = 0

        f_number = 0

    # normalization [-1,+1]
    minfr = np.ones(len(feature[0]))
    mxfr = np.ones(len(feature[0]))
    for norm in range(len(feature[0])):
        minfr[norm] = min(feature[:, norm])
        mxfr[norm] = max(feature[:, norm])

    featurenorm = np.empty((len(feature), len(feature[0])))
    for norm in range(len(feature[0])):
        if (round(minfr[norm],7)== round(mxfr[norm],7)):
            featurenorm[:, norm] = feature[:, norm] - feature[:, norm]
        else:
            featurenorm[:, norm] =(((feature[:, norm] - minfr[norm]) / (mxfr[norm] - minfr[norm])) - 0.5) * 2 

    # delete fix and nan column
    [row, col] = np.where(np.isnan(featurenorm))
    featureout = np.empty((len(featurenorm),0))
    for i in range(len(featurenorm[0])):
        min1 = min(featurenorm[:, i])
        max1 = max(featurenorm[:, i])
        if (min1 < max1 and np.isin(i,col) == 0):
            temp = featurenorm[:, i]
            featureout = np.append(featureout,temp[:,None],1)

    X = featureout
    Y = label
    Y = Y.T

    yptrain = []
    ytrain = []

    yptest = []
    ytest = []

    ypval = []
    yval = []

    #nested cross validation
    cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)
    for trainvalindex, testIdx in cv_outer.split(X):
        Xtrainvalidation = X[trainvalindex,:]
        Ytrainvalidation = Y[trainvalindex]

        Xtest = X[testIdx,:]
        Ytest = Y[testIdx]
        Ytest = Ytest.T

        mdl = NeighborhoodComponentsAnalysis(tol = 0.02)
        mdl.fit(Xtrainvalidation, Ytrainvalidation)
        
        cv_inner = KFold(n_splits=4, shuffle=True, random_state=1)
        bestvalidationacc = 0
    
        for trainindex, valIdx in cv_inner.split(Xtrainvalidation):
            
            Xtrain = Xtrainvalidation[trainindex,:]
            Ytrain = Ytrainvalidation[trainindex]
            Ytrain = Ytrain.T
            Xval = Xtrainvalidation[valIdx,:]
            Yval = Ytrainvalidation[valIdx]
            Yval = Yval.T

            mdl1 = NeighborhoodComponentsAnalysis(tol = 0.02)
            mdl1.fit(Xtrain, Ytrain)
            
            model = SVC(kernel = 'linear')
            model.fit(mdl1.transform(Xtrain), Ytrain)
            
            ytrainp = model.predict(mdl1.transform(Xtrain))
            
            yptrain.append(list(ytrainp))
            ytrain.append(list(Ytrain))

            yvalp = model.predict(mdl1.transform(Xval))
            
            ypval.append(list(yvalp))
            yval.append(list(Yval))
        
            del model, mdl1

        modelout = SVC(kernel = 'linear')
        modelout.fit(mdl.transform(Xtrainvalidation), Ytrainvalidation)
        ytestp = modelout.predict(mdl.transform(Xtest))
        
        yptest.append(list(ytestp))
        ytest.append(list(Ytest))
        
        del modelout, mdl
          
    accvaltest = Evaluated(ytest,yptest,1)
    accvaltrain = Evaluated(ytrain,yptrain,1)
    accvalvalidation = Evaluated(yval,ypval,1)

    out = 1 - accvalvalidation # validation error
    
    global ACCVAL, ACCTRAIN, ACCTEST, fvalbest

    if(out < fvalbest):
        fvalbest = out
        chvalbest = particle
        ACCTRAIN.append(accvaltrain) 
        ACCTEST.append(accvaltest)
        ACCVAL.append(accvalvalidation) 
        print(f"The best validation accuracy ever achieved is {accvalvalidation}")
    else:
        ACCTRAIN.append(ACCTRAIN[FES - 1]) 
        ACCTEST.append(ACCTEST[FES - 1])
        ACCVAL.append(ACCVAL[FES - 1]) 
        
    FES = FES + 1
    
    return out



(200,)
